# STDP Network (Moderate)
This task will create a simple network involving STDP learning rules.

Set up the simulator with a 1.0ms time step.

In [ ]:
import spynnaker.pyNN as p
p.setup(1.0)

Add two single-neuron LIF populations, called "pre" and "post".  Record the spikes from these populations.

In [ ]:
pre = p.Population(1, p.IF_curr_exp())
post = p.Population(1, p.IF_curr_exp())
pre.record("spikes")
post.record("spikes")

Create a Spike Pair rule with a tau_plus of 10.0 and a tau_minus of 1.0ms and an A_plus and A_minus of 0.5

In [ ]:
timing = p.SpikePairRule(tau_plus=10.0, tau_minus=1.0, A_plus=0.5, A_minus=0.5)

Create an Additive Weight Dependence with a minimum weight of 0 and a maximum of 5.0.

In [ ]:
weight = p.AdditiveWeightDependence(w_max=5.0, w_min=0.0)

Create a Projection with a OneToOneConnector and an STDP Mechanism type using the rules above, with a initial weight of 0.

In [ ]:
proj = p.Projection(pre, post, p.OneToOneConnector(), p.STDPMechanism(timing_dependence=timing, weight_dependence=weight, weight=0.0, delay=5.0))

Create a SpikeSourceArray population called "pre_stim" which will be used to stimulate the "pre" neurons.  Set this up to spike at the following times: [0, 10, 30, 40, 50, 60, 70, 80, 90, 110, 120].  Note the gaps in the times!  These will be used to show that the post-neuron is initially not connected to the pre-neuron (it has a weight of 0), but becomes connected after training.

In [ ]:
pre_stim = p.Population(1, p.SpikeSourceArray([0, 10, 30, 40, 50, 60, 70, 80, 90, 110, 120]))

Create a SpikeSourceArray population called "post_stim" which will be used to stimulate the "post" neurons.  Set this up to spike 2 seconds after the neurons in pre_stim, but only for the times between 30 and 90 (remember, the other times are to test the results).

In [ ]:
post_stim = p.Population(1, p.SpikeSourceArray([32, 42, 52, 62, 72, 82, 92]))

Add static projections from "pre_stim" to "pre" and "post_stim" to "post".  Note that a weight of 5.0 should be enough to force an IF_curr_exp neuron to fire with the default parameters.

In [ ]:
p.Projection(pre_stim, pre, p.OneToOneConnector(), p.StaticSynapse(weight=5.0))
p.Projection(post_stim, post, p.OneToOneConnector(), p.StaticSynapse(weight=5.0))

Run the network for 150ms.

In [ ]:
p.run(150)

Extract and plot the spike times.

In [ ]:
pre_spikes = pre.get_data("spikes")
post_spikes = post.get_data("spikes")

In [ ]:
from pyNN.utility.plotting import Figure, Panel
Figure(Panel(pre_spikes.segments[0].spiketrains),
       Panel(post_spikes.segments[0].spiketrains))

Extract and print the weights of the STDP projection.

In [ ]:
weights = proj.get(["weight"], "list")
p.end()
print(weights)

You should be able to see that the weights have changed from the starting values, and that by the end of
the simulation, the second neuron should spike shortly after the first neuron.

## Extension
Alter the parameters of the STDP connection, and the relative timing of the spikes. 

## Extension 2
Try starting with a large initial weight and see if you can get the weight to reduce using the relative timing of the spikes.  You may need to adjust the tau_minus and tau_plus parameters in the above to avoid unwanted effects.